In [1]:
# Import the dependencies.
import pandas as pd
# os for opening file
import os
# gmaps for google maps api
import gmaps
import requests
# API key
from config import g_key2

In [2]:
# Save file path to variable
file_to_load = os.path.join("Resources","WeatherPy_challenge.csv")

In [3]:
# Create DataFrme from csv
vacation_df = pd.read_csv(file_to_load)

In [4]:
# Check df was properly imported from csv
vacation_df.head(1)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
0,0,Busselton,AU,2020-04-01 23:00:27,-33.65,115.33,57.99,67,94,11.52,overcast clouds,Clouds,0.0,0.0


In [5]:
# Explore the data, check data types
vacation_df.dtypes

Unnamed: 0                int64
City                     object
Country                  object
Date                     object
Lat                     float64
Lng                     float64
Max Temp                float64
Humidity                  int64
Cloudiness                int64
Wind Speed              float64
Current Weather          object
Type of Weather          object
Rain Inches(last 3h)    float64
Snow Inches(last 3h)    float64
dtype: object

In [6]:
# Explore the data, check for null values
vacation_df.notnull().sum()

Unnamed: 0              575
City                    575
Country                 573
Date                    575
Lat                     575
Lng                     575
Max Temp                575
Humidity                575
Cloudiness              575
Wind Speed              575
Current Weather         575
Type of Weather         575
Rain Inches(last 3h)    575
Snow Inches(last 3h)    575
dtype: int64

In [7]:
# Drop rows with the two nan values in the dataset
vacation_df = vacation_df.dropna()

In [8]:
# Confirm rows were dropped
vacation_df.notnull().sum()

Unnamed: 0              573
City                    573
Country                 573
Date                    573
Lat                     573
Lng                     573
Max Temp                573
Humidity                573
Cloudiness              573
Wind Speed              573
Current Weather         573
Type of Weather         573
Rain Inches(last 3h)    573
Snow Inches(last 3h)    573
dtype: int64

In [9]:
# Get users minimum and maximum temperatures for their trip
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 95


In [10]:
# Function that takes either yes or no from user
def get_choice(choices, precip_option):
  choice = ""
  while choice not in choices:
      choice = input(f"Would you like it to be {precip_option} on your vacation? [%s]:" % ", ".join(choices))
  return choice
rain_option = get_choice(["yes","no"], "raining")
snow_option = get_choice(["yes","no"], "snowing")

Would you like it to be raining on your vacation? [yes, no]: no
Would you like it to be snowing on your vacation? [yes, no]: no


In [11]:

# Get the locations with temperatures within user defined params
vacation_temp_df = vacation_df.loc[(vacation_df['Max Temp'] >= min_temp)&(vacation_df['Max Temp'] <=max_temp)]

In [12]:
vacation_temp_df.count()

Unnamed: 0              270
City                    270
Country                 270
Date                    270
Lat                     270
Lng                     270
Max Temp                270
Humidity                270
Cloudiness              270
Wind Speed              270
Current Weather         270
Type of Weather         270
Rain Inches(last 3h)    270
Snow Inches(last 3h)    270
dtype: int64

In [13]:
# Set variables to be passed to loc 
rain = "Rain"
snow = "Snow"
if rain_option == "yes":
    vacation_weather_df_1 = vacation_temp_df.loc[(vacation_temp_df['Type of Weather'] == rain)] 
else:
    vacation_weather_df_1 = vacation_temp_df.loc[(vacation_temp_df['Type of Weather'] != rain)]

if snow_option == "yes":
    vacation_weather_df_2 = vacation_weather_df_1.loc[(vacation_weather_df_1['Type of Weather']== snow)]
else:
     vacation_weather_df_2 = vacation_weather_df_1.loc[(vacation_weather_df_1['Type of Weather']!= snow)]
        


In [14]:
vacation_weather_df_2.count()

Unnamed: 0              184
City                    184
Country                 184
Date                    184
Lat                     184
Lng                     184
Max Temp                184
Humidity                184
Cloudiness              184
Wind Speed              184
Current Weather         184
Type of Weather         184
Rain Inches(last 3h)    184
Snow Inches(last 3h)    184
dtype: int64

In [15]:
vacation_weather_df_2.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Type of Weather,Rain Inches(last 3h),Snow Inches(last 3h)
3,3,Hilo,US,2020-04-01 23:00:28,19.73,-155.09,80.60,57,1,5.82,clear sky,Clear,0.0,0.0
5,5,Benguela,AO,2020-04-01 23:00:29,-12.58,13.41,76.78,83,43,3.31,scattered clouds,Clouds,0.0,0.0
14,14,Vaini,TO,2020-04-01 23:00:31,-21.20,-175.20,84.20,74,75,4.70,broken clouds,Clouds,0.0,0.0
15,15,Villa Bruzual,VE,2020-04-01 23:00:31,9.33,-69.12,89.29,38,88,2.57,overcast clouds,Clouds,0.0,0.0
16,16,Puerto Ayora,EC,2020-04-01 23:00:32,-0.74,-90.35,84.20,70,20,10.29,few clouds,Clouds,0.0,0.0


In [16]:
hotel_df = vacation_weather_df_2[['City', 'Country', 'Max Temp', 'Lat', 'Lng', 'Current Weather']].copy()

In [17]:
hotel_df.head(3)

,City,Country,Max Temp,Lat,Lng,Current Weather
3,Hilo,US,80.60,19.73,-155.09,clear sky
5,Benguela,AO,76.78,-12.58,13.41,scattered clouds
14,Vaini,TO,84.20,-21.20,-175.20,broken clouds


In [18]:
params = {'radius': 5000, 'type': 'lodging', 'key': g_key2}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
print("fin")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
fin


In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Conditions</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
gmaps.configure(api_key=g_key2)
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
# Store the DataFrame Row.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
3,Hilo,US,80.60,19.73,-155.09,clear sky,Hilo Hawaiian Hotel
5,Benguela,AO,76.78,-12.58,13.41,scattered clouds,Hotel Praia Morena
14,Vaini,TO,84.20,-21.20,-175.20,broken clouds,Keleti Beach Resort
15,Villa Bruzual,VE,89.29,9.33,-69.12,overcast clouds,Hotel Turen
16,Puerto Ayora,EC,84.20,-0.74,-90.35,few clouds,Finch Bay Galapagos Hotel
...,...,...,...,...,...,...,...
559,Avera,US,66.20,33.19,-82.53,broken clouds,NaN
560,Loa Janan,ID,80.04,-0.58,117.09,broken clouds,Asrama Polisi Polsek Loajanan Res Kukar
562,Abu Dhabi,AE,75.00,24.47,54.37,few clouds,Beach Rotana
566,Palauig,PH,85.23,15.44,119.90,few clouds,Myhousebythesea


In [22]:
save_path = os.path.join('Resources','WeatherPy_vacation.csv')
hotel_df.to_csv(save_path)